In [5]:
import anndata
import pandas as pd
import scipy.io
def import_matrix_as_anndata(matrix_path, barcodes_path, genes_path):
    df_barcodes = pd.read_csv(
        barcodes_path, index_col=0, header=None, names=['barcode']
    )
    df_genes = pd.read_csv(
        genes_path, header=None, index_col=0, names=['gene_id'], sep='\t'
    )
    df_genes.index = df_genes.index.str.split('.').str[
        0]  # slice off version number
    return anndata.AnnData(
        X=scipy.io.mmread(matrix_path).tocsr(), obs=df_barcodes, var=df_genes
    )


In [6]:
adata_spliced = import_matrix_as_anndata('spliced.mtx', 'spliced.barcodes.txt', 'spliced.genes.txt')
adata_unspliced = import_matrix_as_anndata('unspliced.mtx', 'unspliced.barcodes.txt', 'unspliced.genes.txt')

In [7]:
print(adata_spliced)
print(adata_unspliced)

AnnData object with n_obs × n_vars = 278 × 17 
AnnData object with n_obs × n_vars = 278 × 17 


In [8]:
t2g = pd.read_csv('../../t2g.txt', header=None, sep='\t', names=['tid', 'gid', 'gene'])

In [17]:
adata_spliced.obs.index

Index(['AAACGGGAGTCATGCT', 'AAAGCAAGTTCCTCCA', 'AAATGCCGTAGCGCTC',
       'AACACGTGTATTAGCC', 'AACCATGGTTTAAGCC', 'AACCATGGTTTGGGCC',
       'AAGACCTAGATGCGAC', 'AAGCCGCGTTACGACT', 'AAGGAGCAGGCAGGTT',
       'AAGGCAGAGTAGCGGT',
       ...
       'TTGCCGTAGAAGGGTA', 'TTGCCGTGTATCGCAT', 'TTGCGTCCAGGGTATG',
       'TTGGAACAGAATGTGT', 'TTGTAGGAGGACAGCT', 'TTGTAGGGTCTAGTCA',
       'TTTACTGAGTACTTGC', 'TTTATGCCACCTCGTT', 'TTTGCGCCACGGACAA',
       'TTTGGTTTCCGTACAA'],
      dtype='object', name='barcode', length=278)

In [18]:
adata_spliced = adata_spliced[adata_spliced.obs.index.isin(adata_unspliced.obs.index)]
adata_unspliced = adata_unspliced[adata_unspliced.obs.index.isin(adata_spliced.obs.index)]

In [19]:
adata_spliced = adata_spliced[:, adata_spliced.var.index.isin(adata_unspliced.var.index)]
adata_unspliced = adata_unspliced[:, adata_unspliced.var.index.isin(adata_spliced.var.index)]

In [20]:
adata_spliced

View of AnnData object with n_obs × n_vars = 278 × 17 

In [21]:
adata_spliced.layers["spliced"] = adata_spliced.X
adata_spliced.layers["unspliced"] = adata_unspliced.X

In [22]:
adata_spliced

AnnData object with n_obs × n_vars = 278 × 17 
    layers: 'spliced', 'unspliced'

In [ ]:
adata_spliced.write